In [1]:
using PyPlot
using JLD
using LinearAlgebra
using SparseArrays
using BenchmarkTools
import Base.Threads: nthreads, @spawn
using LoopVectorization

nthreads()

true

In [18]:
function k(X,x,sigma)
    s = zero(eltype(x))
    @simd for i in eachindex(x,X)
            s += (X[i] - x[i])^2
        end
    return exp(-s/sigma^2)
end

function U_block_in_place(partA,partSol,sigma)
     for i in 1:length(partSol)
         for j in 1:i
             partA[j,i]=k(partSol[i],partSol[j],sigma)
            end
        end
end

#=
function U_block_in_place(partA,partSol,sigma)
    @turbo for i in 1:length(partSol)
        x = partSol[i]
        for j in 1:i
            X = partSol[j]
            s = zero(eltype(partSol))
            for k in eachindex(x, X)
                s += (X[k] - x[k])^2
            end
            partA[j,i] = exp(-s/sigma^2)
        end
    end
end
=#
function full_block_in_place(partA,partSol1,partSol2,sigma)
     for i in 1:length(partSol2)
          for j in 1:length(partSol1)
              partA[j,i]=k(partSol1[j],partSol2[i],sigma)
        end
    end
end

function K_com_inplace(solucionesn,sigma,iOO,iOF,iFO,iFF)
    A=zeros(length(solucionesn),length(solucionesn))
    D1=@spawn U_block_in_place(@view(A[iOO,iOO]),deepcopy(solucionesn[iOO]),sigma)
    D2=@spawn U_block_in_place(@view(A[iOF,iOF]),deepcopy(solucionesn[iOF]),sigma)
    D3=@spawn U_block_in_place(@view(A[iFO,iFO]),deepcopy(solucionesn[iFO]),sigma)
    D4=@spawn U_block_in_place(@view(A[iFF,iFF]),deepcopy(solucionesn[iFF]),sigma)
    D5=@spawn full_block_in_place(@view(A[iOO,iOF]),deepcopy(solucionesn[iOO]),deepcopy(solucionesn[iOF]),sigma)
    D6=@spawn full_block_in_place(@view(A[iOF,iFO]),deepcopy(solucionesn[iOF]),deepcopy(solucionesn[iFO]),sigma)
    D7=@spawn full_block_in_place(@view(A[iFO,iFF]),deepcopy(solucionesn[iFO]),deepcopy(solucionesn[iFF]),sigma)
    D8=@spawn full_block_in_place(@view(A[iOO,iFO]),deepcopy(solucionesn[iOO]),deepcopy(solucionesn[iFO]),sigma)
    D9=@spawn full_block_in_place(@view(A[iOF,iFF]),deepcopy(solucionesn[iOF]),deepcopy(solucionesn[iFF]),sigma)
    D10=@spawn full_block_in_place(@view(A[iOO,iFF]),deepcopy(solucionesn[iOO]),deepcopy(solucionesn[iFF]),sigma)
    
    rD1=fetch(D1)
    rD2=fetch(D2)
    rD3=fetch(D3)
    rD4=fetch(D4)
    rD5=fetch(D5)
    rD6=fetch(D6)
    rD7=fetch(D7)
    rD8=fetch(D8)
    rD9=fetch(D9)
    rD10=fetch(D10)
    return Symmetric(A)
end

soln=Matrix{Float64}[]
for i in 1:10000
    append!(soln,[rand(50,50)])
end
iOO=1:2550
iOF=2551:5026
iFO=5027:7530
iFF=7531:10000

7531:10000

In [19]:
@time K,K0= K_com_inplace(soln,1.0,iOO,iOF,iFO,iFF)

 34.509727 seconds (426.00 k allocations: 1.511 GiB, 0.03% gc time, 0.78% compilation time)


10000×10000 Symmetric{Float64, Matrix{Float64}}:
 1.0           1.2516e-177   3.14263e-176  …  3.62192e-176  1.99633e-191
 1.2516e-177   1.0           3.84638e-180     1.30844e-177  1.52978e-187
 3.14263e-176  3.84638e-180  1.0              3.64235e-176  1.3855e-176
 7.51634e-180  2.53221e-180  2.18725e-168     7.03099e-169  5.58571e-178
 2.60839e-180  1.48419e-180  3.72694e-181     2.93247e-184  1.20514e-181
 3.92839e-183  1.29207e-179  2.86419e-175  …  6.49289e-173  8.27141e-178
 1.18268e-174  2.7572e-182   1.12295e-175     4.25116e-178  9.24792e-183
 3.22808e-181  1.03418e-175  5.06047e-176     2.81503e-171  1.15245e-185
 2.96781e-178  9.51342e-184  2.275e-178       9.39906e-183  4.26962e-178
 3.72488e-175  2.70507e-186  2.06693e-176     1.00461e-177  5.64167e-181
 1.08612e-178  3.04708e-172  2.70473e-177  …  2.08793e-184  1.54138e-179
 7.04851e-180  2.35434e-183  1.70045e-181     1.02731e-176  3.40516e-187
 4.16917e-183  9.6897e-182   3.91269e-182     1.81648e-186  1.28308e-187
 ⋮ 